In [4]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('tsunami_dataset.csv')

In [7]:
# Display information about the DataFrame
print("--- DataFrame Info ---")
df.info()

# Display descriptive statistics of numerical columns
print("\n--- Descriptive Statistics ---")
df.describe()

# Check for missing values in each column
print("\n--- Missing Values Count ---")
print(df.isnull().sum())

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2259 entries, 0 to 2258
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        2259 non-null   int64  
 1   YEAR                      2259 non-null   int64  
 2   MONTH                     2155 non-null   float64
 3   DAY                       2082 non-null   float64
 4   HOUR                      1308 non-null   float64
 5   MINUTE                    1235 non-null   float64
 6   LATITUDE                  2259 non-null   float64
 7   LONGITUDE                 2259 non-null   float64
 8   LOCATION_NAME             2250 non-null   object 
 9   COUNTRY                   2259 non-null   object 
 10  REGION                    2258 non-null   object 
 11  CAUSE                     2258 non-null   object 
 12  EVENT_VALIDITY            2259 non-null   object 
 13  EQ_MAGNITUDE              1474 non-null 